# "Bin" statistics & histogram

Statistics "bin" are counted on the sample of a single continuous variable.
Firstly, it divides the range of values into bins (sequential, non-overlapping sections),
and then it counts the number of observations in each bin.
It's weighted, it means the weighted count for each bin is calculated
(each element within a bin counted along with its weight).
It's really important to carefully choose a bin-constructing method
(for example, by the exact number of bins or by their width).
This decision has a big impact on how the data is shown and studied.
It makes sure that the way the data is shown is natural to understand and gives a true picture of the information.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

Binning is commonly used in statistics and data analysis
to simplify complex data sets and make them easier to interpret.
Histogram (or any other plot with "bin" statistics) helps to give an overview of the sample distribution.

## Arguments

* Input (mandatory):
    - `x` — numeric sample on which the statistics are calculated
* Weights (optional):
    - `weights` — set of weights of the same size as the input sample.
      `null` (by default) means all weights are equal to `1.0` and the weighted count is equal to the normal one
* Parameters (optional):
    - `binsOption: BinsOption` — specifies either the number of bins or their width:
        - `BinsOption.byNumber(n: Int)` — values are divided into `n` bins (bins width is derived)
        - `BinsOption.byWidth(width: Double)` — values are divided into bins of width `width`
          (the number of bins is derived)
    - `binsAlign: BinsAlign` — specifies bins aligning:
        - `BinsAlign.center(pos: Double)` — bins are aligned by centering bin in `pos`
        - `BinsAlign.boundary(pos: Double)` — bins are aligned by boundary between two bins in `pos`
        - `BinsAlign.none()` — no aligning

### Generalized signature

The specific signature depends on the function,
but all functions related to "bin" statistic
(which will be discussed further below - different variations of `statBin()`, `histogram()`)
have approximately the same signature with the arguments above:

```
statBinArgs := 
   x, 
   weights = null, 
   binsOption: BinsOption = BinsOption.byNumber(20), 
   binsAlign: BinsAlign = BinsAlign.center(0.0)
```

The possible types of `x` and `weights` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name                 | type   | description                                              |
|----------------------|--------|----------------------------------------------------------|
| Stat.x               | Double | Center of bin                                            |
| Stat.count           | Int    | Number of observations in this bin                       |
| Stat.countWeighted   | Double | Weighted count (sum of observations weights in this bin) |
| Stat.density         | Double | Empirically estimated density in this bin                |
| Stat.densityWeighted | Double | Weighted density                                         |

## StatBin plots

In [1]:
%useLatestDescriptors
%use kandy
%use dataframe

import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import org.jetbrains.kotlinx.statistics.dataframe.stat.mean

In [2]:
// Generate sample from normal distribution
val depthList = NormalDistribution(500.0, 100.0).sample(1000).toList()
// Generate sample from uniform distribution
val coeffList = UniformRealDistribution(0.0, 1.0).sample(1000).toList()
// gather them into the DataFrame
val df = dataFrameOf(
    "depth" to depthList,
    "coeff" to coeffList
)
df.head()

depth,coeff
"418,998096","0,139201"
"517,386447","0,547850"
"522,005359","0,678063"
"646,213794","0,741511"
"523,485249","0,264879"


`df` has a signature

| depth | coeff |
|-------|-------|

Let's take a look at `StatBin` output DataFrame:

In [3]:
df.statBin("depth", "coeff", binsOption = BinsOption.byNumber(10))

Stat,,,,
x,count,countWeighted,density,densityWeighted
"169,600849",3,"2,363360","0,000044","0,000070"
"237,441189",12,"5,669811","0,000177","0,000169"
"305,281529",49,"26,187534","0,000722","0,000780"
"373,121868",128,"58,333240","0,001887","0,001738"
"440,962208",219,"102,736608","0,003228","0,003060"
"508,802548",269,"133,482524","0,003965","0,003976"
"576,642887",193,"101,973681","0,002845","0,003038"
"644,483227",96,"47,791460","0,001415","0,001424"
"712,323567",27,"14,697443","0,000398","0,000438"


It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>count</th>
      <th>countWeighted</th>
      <th>density</th>
      <th>densityWeighted</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statBin`, each row corresponds to one bin.
`Stat.x` is the column with the centers of the bins.
`Stat.count` contains the number of observations in the bin.
`Stat.countWeighted` — weighted version of `count`.
There are also `Stat.density` and `Stat.densityWeighted`.
They contain empirically estimated density (both normal and weighted)
of the sample in the points corresponding to the centers of bins.

`DataFrame` with "bin" statistics is called `StatBinFrame`

### `statBin` context transform

`statBin(statBinArgs) { /*new plotting context*/ }` modifies a plotting context — instead of original data
(no matter was it empty or not) new `StatBin` dataset (calculated on given arguments.
Inputs and weights can be provided as `Iterable` or as dataset column reference — by name as a `String`,
as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statBin` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [4]:
plot {
    statBin(depthList, binsAlign = BinsAlign.center(500.0)) {
        // new `StatBin` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.count)
            fillColor = Color.RED
            alpha = 0.5
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2zoVt8"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[160.79830159113374,194.71847143202035,228.63864127290702,262.55881111379364,296.47898095468025,330.39915079556687,364.31932063645354,398.23949047734015,432.15966031822677,466.0798301591134,500.0,533.9201698408867,567.8403396817732,601.76050952266,635.6806793635466,669.6008492044332,703.5210190453198,737.4411888862064,771.361358727093,805.2815285679796],
"count":[2.0,1.0,5.0,9.0,19.0,36.0,55.0,79.0,123.0,114.0,120.0,152.0,106.0,68.0,56.0,29.0,17.0,6.0,2.0,1.0]
},
"sampling":"none",
"alpha":0.5,
"position":"identity",
"geom":"area",
"fill":"#ee6666"
}]
};
 var plotContainer = document.getElementById("2zoVt8");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.411735619708892 339.6892230576441 L24.411735619708892 339.6892230576441 L50.10829942992875 341.84461152882204 L75.80486324014866 333.22305764411027 L101.50142705036852 324.6015037593985 L127.19799086058838 303.04761904761904 L152.89455467080825 266.406015037594 L178.59111848102816 225.45363408521303 L204.28768229124802 173.72431077694236 L229.98424610146787 78.88721804511277 L255.68080991168773 98.28571428571428 L281.3773737219076 85.35338345864659 L307.07393753212756 16.38095238095235 L332.7705013423473 115.52882205513782 L358.4670651525673 197.43358395989975 L384.16362896278713 223.29824561403507 L409.860192773007 281.4937343358396 L435.55675658322684 307.35839598997495 L461.25332039344676 331.0676691729323 L486.94988420366656 339.6892230576441 L512.6464480138865 341.84461152882204 L512.6464480138865 344.0 L486.94988420366656 344.0 L461.25332039344676 344.0 L435.55675658322684 344.0 L409.860192773007 344.0 L384.16362896278713 344.0 L358.4670651525673 344.0 L332.7705013423473 344.0 L307.07393753212756 344.0 L281.3773737219076 344.0 L255.68080991168773 344.0 L229.98424610146787 344.0 L204.28768229124802 344.0 L178.59111848102816 344.0 L152.89455467080825 344.0 L127.19799086058838 344.0 L101.50142705036852 344.0 L75.80486324014866 344.0 L50.10829942992875 344.0 L24.411735619708892 344.0 Z" fill="rgb(238,102,102)" stroke-width="1.0" fill-opacity="0.5019607843137255">
 
 
 
 <path d="M24.411735619708892 339.6892230576441 L24.411735619708892 339.6892230576441 L50.10829942992875 341.84461152882204 L75.80486324014866 333.22305764411027 L101.50142705036852 324.6015037593985 L127.19799086058838 303.04761904761904 L152.89455467080825 266.406015037594 L178.59111848102816 225.45363408521303 L204.28768229124802 173.72431077694236 L229.98424610146787 78.88721804511277 L255.68080991168773 98.28571428571428 L281.3773737219076 85.35338345864659 L307.07393753212756 16.38095238095235 L332.7705013423473 115.52882205513782 L358.4670651525673 197.43358395989975 L384.16362896278713 223.29824561403507 L409.860192773007 281.4937343358396 L435.55675658322684 307.35839598997495 L461.25332039344676 331.0676691729323 L486.94988420366656 339.6892230576441 L512.6464480138865 341.84461152882204 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)

### Histogram layer

A histogram is a statistical chart that serves to visually approximate the distribution of a numerical variable.
It's a bar plot where each bar is representing a bin: its x coordinate is corresponding to bin range and y to count.
So basically, we can build a histogram with `statBin` as follows:

In [5]:
val statBinBarsPlot = df.plot {
    statBin("depth") {
        bars {
            x(Stat.x)
            y(Stat.count)
        }
    }
    layout.title = "`statBin` + `bars`"
}
statBinBarsPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Oz45hj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"`statBin` + `bars`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[2.0,1.0,4.0,8.0,15.0,34.0,44.0,84.0,106.0,113.0,122.0,147.0,117.0,76.0,63.0,33.0,20.0,7.0,3.0,1.0],
"x":[152.64076428398982,186.56093412487644,220.4811039657631,254.40127380664973,288.32144364753634,322.24161348842296,356.1617833293096,390.08195317019624,424.00212301108286,457.9222928519695,491.8424626928561,525.7626325337428,559.6828023746293,593.602972215516,627.5231420564027,661.4433118972893,695.3634817381759,729.2836515790625,763.2038214199491,797.1239912608357]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("Oz45hj");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `statBin` + `bars` 
 
 
 
 
 count 
 
 
 
 
 x

But we can do it even faster with `histogram(statBinArgs)` method:

In [6]:
val histogramPlot = plot {
    histogram(depthList)
    layout.title = "`histogram`"
}
histogramPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vxRc8Z"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"`histogram`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[2.0,1.0,4.0,8.0,15.0,34.0,44.0,84.0,106.0,113.0,122.0,147.0,117.0,76.0,63.0,33.0,20.0,7.0,3.0,1.0],
"x":[152.64076428398982,186.56093412487644,220.4811039657631,254.40127380664973,288.32144364753634,322.24161348842296,356.1617833293096,390.08195317019624,424.00212301108286,457.9222928519695,491.8424626928561,525.7626325337428,559.6828023746293,593.602972215516,627.5231420564027,661.4433118972893,695.3634817381759,729.2836515790625,763.2038214199491,797.1239912608357]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("vxRc8Z");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `histogram` 
 
 
 
 
 count 
 
 
 
 
 x

Let's compare them:

In [7]:
plotGrid(listOf(statBinBarsPlot, histogramPlot))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="PpyXls"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`statBin` + `bars`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[152.64076428398982,186.56093412487644,220.4811039657631,254.40127380664973,288.32144364753634,322.24161348842296,356.1617833293096,390.08195317019624,424.00212301108286,457.9222928519695,491.8424626928561,525.7626325337428,559.6828023746293,593.602972215516,627.5231420564027,661.4433118972893,695.3634817381759,729.2836515790625,763.2038214199491,797.1239912608357],
"count":[2.0,1.0,4.0,8.0,15.0,34.0,44.0,84.0,106.0,113.0,122.0,147.0,117.0,76.0,63.0,33.0,20.0,7.0,3.0,1.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
},{
"ggtitle":{
"text":"`histogram`"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[152.64076428398982,186.56093412487644,220.4811039657631,254.40127380664973,288.32144364753634,322.24161348842296,356.1617833293096,390.08195317019624,424.00212301108286,457.9222928519695,491.8424626928561,525.7626325337428,559.6828023746293,593.602972215516,627.5231420564027,661.4433118972893,695.3634817381759,729.2836515790625,763.2038214199491,797.1239912608357],
"count":[2.0,1.0,4.0,8.0,15.0,34.0,44.0,84.0,106.0,113.0,122.0,147.0,117.0,76.0,63.0,33.0,20.0,7.0,3.0,1.0]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
}],
"kind":"subplots"
};
 var plotContainer = document.getElementById("PpyXls");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `statBin` + `bars` 
 
 
 
 
 count 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 `histogram` 
 
 
 
 
 count 
 
 
 
 
 x

These two plots are identical.
Indeed, `histogram` just uses `statBin` and `bars` and performs coordinates mappings under the hood.
And we can customize histogram layer: `histogram()` optionally opens a new context,
where we can configure bars (as in the usual context opened by `bars { ... }`) —
even change coordinate mappings from default ones.
`StatBin` dataset of histogram is also can be accessed here.

In [8]:
df.plot {
    histogram(depth, binsAlign = BinsAlign.center(500.0)) {
        // Change a column mapped on `y` to `Stat.density`
        y(Stat.density)
        // Filling color depends on `density` statistic
        fillColor(Stat.density) {
            scale = continuous(Color.YELLOW..Color.RED)
        }
        borderLine.color = Color.BLACK
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cWRgcb"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#fac858",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"fill":"density"
},
"stat":"identity",
"data":{
"density":[5.8961968922373846E-5,2.9480984461186923E-5,1.4740492230593463E-4,2.653288601506823E-4,5.601387047625515E-4,0.0010613154406027292,0.0016214541453652808,0.002328997772433767,0.0036261610887259915,0.0033608322285753095,0.003537718135342431,0.004481109638100412,0.003124984352885814,0.002004706943360711,0.0016509351298264678,8.549485493744209E-4,5.011767358401777E-4,1.7688590676712154E-4,5.8961968922373846E-5,2.9480984461186923E-5],
"x":[160.79830159113374,194.71847143202035,228.63864127290702,262.55881111379364,296.47898095468025,330.39915079556687,364.31932063645354,398.23949047734015,432.15966031822677,466.0798301591134,500.0,533.9201698408867,567.8403396817732,601.76050952266,635.6806793635466,669.6008492044332,703.5210190453198,737.4411888862064,771.361358727093,805.2815285679796]
},
"color":"#000000",
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("cWRgcb");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000 
 
 
 
 
 
 
 0.0005 
 
 
 
 
 
 
 0.0010 
 
 
 
 
 
 
 0.0015 
 
 
 
 
 
 
 0.0020 
 
 
 
 
 
 
 0.0025 
 
 
 
 
 
 
 0.0030 
 
 
 
 
 
 
 0.0035 
 
 
 
 
 
 
 0.0040 
 
 
 
 
 
 
 0.0045 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 density 
 
 
 
 
 depth 
 
 
 
 
 
 
 
 
 density 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.001 
 
 
 
 
 
 
 
 
 0.002 
 
 
 
 
 
 
 
 
 0.003 
 
 
 
 
 
 
 
 
 0.004

If we specify weights, `Stat.countWeighted` is mapped to `y` by default:

In [9]:
df.plot {
    // Count sample mean
    val mean = depth.mean()
    // Add weighted histogram
    histogram(depth, coeff, binsOption = BinsOption.byNumber(10), binsAlign = BinsAlign.boundary(mean))
    // We can add other layers as well.
    // Let's add a vertical mark line in the mean of sample
    vLine {
        xIntercept.constant(mean)
        tooltips { line("Depth mean: ${String.format("%.2f", mean)}m") }
        color = Color.RED; width = 3.0
    }
    x.axis.name = "depth, m"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="l2bba8"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"depth, m",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"data":{
"x":[190.09162436117862,257.9319640429519,325.77230372472513,393.61264340649836,461.45298308827165,529.2933227700449,597.1336624518182,664.9740021335914,732.8143418153646,800.6546814971379],
"countWeighted":[3.009691745789024,10.27085333274373,34.5629558771382,71.50387710290728,111.11608395700235,133.93011100582186,86.86352827696813,35.04150158873047,7.908045517697261,0.64680620390751]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
},{
"mapping":{
},
"stat":"identity",
"color":"#ee6666",
"size":3.0,
"sampling":"none",
"position":"identity",
"xintercept":495.37315292915827,
"geom":"vline",
"tooltips":{
"lines":["Depth mean: 495,37m"],
"disable_splitting":true
},
"data":{
}
}]
};
 var plotContainer = document.getElementById("l2bba8");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted 
 
 
 
 
 depth, m

### `histogram` plot

`histogram(statBinArgs)` and `DataFrame.histogram(statBinArgs)` are a family of functions for fast plotting a histogram.

In [10]:
histogram(depthList, binsAlign = BinsAlign.center(500.0))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Gx0aPg"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[2.0,1.0,5.0,9.0,19.0,36.0,55.0,79.0,123.0,114.0,120.0,152.0,106.0,68.0,56.0,29.0,17.0,6.0,2.0,1.0],
"x":[160.79830159113374,194.71847143202035,228.63864127290702,262.55881111379364,296.47898095468025,330.39915079556687,364.31932063645354,398.23949047734015,432.15966031822677,466.0798301591134,500.0,533.9201698408867,567.8403396817732,601.76050952266,635.6806793635466,669.6008492044332,703.5210190453198,737.4411888862064,771.361358727093,805.2815285679796]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("Gx0aPg");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 x

In [11]:
df.histogram("depth")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8sNO6H"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"x":[152.64076428398982,186.56093412487644,220.4811039657631,254.40127380664973,288.32144364753634,322.24161348842296,356.1617833293096,390.08195317019624,424.00212301108286,457.9222928519695,491.8424626928561,525.7626325337428,559.6828023746293,593.602972215516,627.5231420564027,661.4433118972893,695.3634817381759,729.2836515790625,763.2038214199491,797.1239912608357],
"count":[2.0,1.0,4.0,8.0,15.0,34.0,44.0,84.0,106.0,113.0,122.0,147.0,117.0,76.0,63.0,33.0,20.0,7.0,3.0,1.0]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("8sNO6H");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 depth

In case you want to provide inputs and weights using column selection DSL, it's a bit different from the usual one —
you should assign `x` input and (optionally) `weight` throw invocation eponymous functions:

In [12]:
df.histogram(binsOption = BinsOption.byNumber(10)) {
    x(depth)
    weight(coeff)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="BCnFv3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"data":{
"x":[169.60084920443313,237.44118888620642,305.28152856797965,373.1218682497529,440.96220793152617,508.80254761329945,576.6428872950727,644.4832269768459,712.3235666586191,780.1639063403924],
"countWeighted":[2.363360417726903,5.669811407379993,26.187533622661373,58.333240020679206,102.73660788161456,133.4825236783065,101.97368122528331,47.791459500353525,14.697443220326273,1.6177936343742174]
},
"sampling":"none",
"position":"identity",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("BCnFv3");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 140 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted 
 
 
 
 
 depth

Histogram plot can be configured with `.configure {}` extension —
it opens a context that combines bars, `StatBin` and plot context.
That means you can configure bars settings, mappings using `StatBin` dataset and any plot adjustments:

In [13]:
df.histogram(binsOption = BinsOption.byNumber(15)) {
    x(depth)
}.configure {
    // Bars + StatBin + PlotBuilder
    // Can't add a new layer
    x.limits = 100..900
    // Can add bar mapping, include on `Stat.*` columns
    fillColor(Stat.count) { scale = continuous(Color.GREEN..Color.RED) }
    // Can configure general plot adjustments
    layout {
        title = "Configured histogram plot"
        size = 600 to 350
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="70jEwt"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Configured histogram plot"
},
"mapping":{
},
"data":{
"x":[158.29412592413757,203.52101904531975,248.7479121665019,293.97480528768403,339.20169840886626,384.4285915300484,429.65548465123055,474.8823777724127,520.1092708935948,565.3361640147771,610.5630571359593,655.7899502571413,701.0168433783235,746.2437364995058,791.4706296206879],
"count":[2.0,3.0,10.0,22.0,57.0,98.0,143.0,158.0,187.0,149.0,85.0,55.0,24.0,6.0,1.0]
},
"ggsize":{
"width":600.0,
"height":350.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"depth",
"limits":[100.0,900.0]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#3ba272",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"depth",
"limits":[100.0,900.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"count"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("70jEwt");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Configured histogram plot 
 
 
 
 
 count 
 
 
 
 
 depth 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 150

## Grouped `staBin`

`statBin` can be applied for grouped data — statistics will be counted on each group independently but with equal bins.
This application returns a new `GroupBy`
dataset with the same keys as the old one but with `StatBin` groups instead of old ones.

In [14]:
// Create two samples from normal distribution with different mean/std
val rangesA = NormalDistribution(500.0, 100.0).sample(5000).toList()
val rangesB = NormalDistribution(400.0, 80.0).sample(5000).toList()

// Gather them into `DataFrame` with "A" and "B" keys in the "category" column
val rangesDF = dataFrameOf(
    "range" to rangesA + rangesB,
    "category" to List(5000) { "A" } + List(5000) { "B" }
)
rangesDF.head(5)

range,category
"630,169591",A
"469,938049",A
"385,775780",A
"332,599287",A
"573,972979",A


It has the following signature:

| range | category |
|-------|----------|

In [15]:
// Group it by "category"
val groupedRangesDF = rangesDF.groupBy { category }
groupedRangesDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-2097151994"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) 

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[range|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [16]:
groupedRangesDF.statBin { x(range) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-2097151988"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) 

After `statBin` applying it's still a `GroupBy` but with different signature of `group` —
all groups have the same signature as usual `DataFrame` after `statBin` applying (i.e. `StatBinFrame`):

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: StaBinFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statBin` transformation within groups, the grouping keys did not change.
Also, all bin centers match — it helps to build grouped histogram.

The plotting process doesn't change much — we do everything the same.

In [17]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="nCCEBi"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"density":[0.0,1.1710669376078378E-5,2.3421338752156756E-5,7.611935094450945E-5,2.517793915856851E-4,5.855334688039188E-4,0.0012003436110480338,0.0019146944429888147,0.002792994646194693,0.003489779474071356,0.00404018093474704,0.0038996529022341,0.003565898825015866,0.0028164159849468494,0.0021840398386386174,0.001276462961992543,6.499421503723499E-4,3.337540772182337E-4,1.0539602438470539E-4,5.855334688039188E-5,6.1958176573445815E-6,6.195817657344581E-5,2.540285239511278E-4,6.381692187064918E-4,0.0014869962377626995,0.0025031103335672105,0.004107827106819457,0.005371773908917752,0.005452319538463231,0.004659254878323125,0.0031722586405604257,0.0018959202031474417,8.364353837415184E-4,3.6555324178333024E-4,1.301121708042362E-4,2.4783270629378326E-5,6.1958176573445815E-6,0.0,6.1958176573445815E-6,0.0],
"x":[124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379,124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379]
},
"sampling":"none",
"position":"identity",
"geom":"area"
}]
};
 var plotContainer = document.getElementById("nCCEBi");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 
 
 
 
 0.001 
 
 
 
 
 
 
 0.002 
 
 
 
 
 
 
 0.003 
 
 
 
 
 
 
 0.004 
 
 
 
 
 
 
 0.005 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.818367630399194 343.6277056277056 L23.818367630399194 343.6277056277056 L48.89033355713518 340.27705627705626 L73.96229948387118 328.73593073593076 L99.03426541060718 305.65367965367966 L124.10623133734315 254.64935064935065 L149.1781972640792 193.5930735930736 L174.2501631908151 97.16883116883122 L199.32212911755113 21.220779220779207 L224.39409504428716 16.380952380952408 L249.46606097102313 64.03463203463201 L274.5380268977591 153.38528138528136 L299.60999282449507 230.0779220779221 L324.6819587512311 293.7402597402597 L349.75392467796706 322.034632034632 L374.82589060470303 336.1818181818182 L399.897856531439 342.5108225108225 L424.96982245817503 343.6277056277056 L450.041788384911 344.0 L475.1137543116471 343.6277056277056 L500.18572023838294 344.0 L500.18572023838294 344.0 L475.1137543116471 344.0 L450.041788384911 344.0 L424.96982245817503 344.0 L399.897856531439 344.0 L374.82589060470303 344.0 L349.75392467796706 344.0 L324.6819587512311 344.0 L299.60999282449507 344.0 L274.5380268977591 344.

As you can see, there are two areas because we have two groups of data.
To distinguish them, we need to add mapping to the filling color from the key.
This is convenient — the key is available in the context

In [18]:
groupedRangesDF.plot {
    statBin(range) {
        area {
            x(Stat.x)
            y(Stat.density)
            // can access "key." columns and create mapping from them
            fillColor(key.category)
            alpha = 0.6
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="HR18XE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"density",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"density":[0.0,1.1710669376078378E-5,2.3421338752156756E-5,7.611935094450945E-5,2.517793915856851E-4,5.855334688039188E-4,0.0012003436110480338,0.0019146944429888147,0.002792994646194693,0.003489779474071356,0.00404018093474704,0.0038996529022341,0.003565898825015866,0.0028164159849468494,0.0021840398386386174,0.001276462961992543,6.499421503723499E-4,3.337540772182337E-4,1.0539602438470539E-4,5.855334688039188E-5,6.1958176573445815E-6,6.195817657344581E-5,2.540285239511278E-4,6.381692187064918E-4,0.0014869962377626995,0.0025031103335672105,0.004107827106819457,0.005371773908917752,0.005452319538463231,0.004659254878323125,0.0031722586405604257,0.0018959202031474417,8.364353837415184E-4,3.6555324178333024E-4,1.301121708042362E-4,2.4783270629378326E-5,6.1958176573445815E-6,0.0,6.1958176573445815E-6,0.0],
"x":[124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379,124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"area"
}]
};
 var plotContainer = document.getElementById("HR18XE");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 
 
 
 
 0.001 
 
 
 
 
 
 
 0.002 
 
 
 
 
 
 
 0.003 
 
 
 
 
 
 
 0.004 
 
 
 
 
 
 
 0.005 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M19.865713083991878 343.6277056277056 L19.865713083991878 343.6277056277056 L40.77699001450965 340.27705627705626 L61.68826694502745 328.73593073593076 L82.59954387554522 305.65367965367966 L103.510820806063 254.64935064935065 L124.42209773658077 193.5930735930736 L145.33337466709855 97.16883116883122 L166.24465159761633 21.220779220779207 L187.1559285281341 16.380952380952408 L208.06720545865193 64.03463203463201 L228.97848238916964 153.38528138528136 L249.8897593196874 230.0779220779221 L270.8010362502052 293.7402597402597 L291.712313180723 322.034632034632 L312.62359011124073 336.1818181818182 L333.5348670417585 342.5108225108225 L354.4461439722763 343.6277056277056 L375.3574209027941 344.0 L396.2686978333119 343.6277056277056 L417.1799747638296 344.0 L417.1799747

The `histogram` layer also works.
Moreover, if we have exactly one grouping key, a mapping from it to `fillColor` will be created by default.

In [19]:
groupedRangesDF.plot {
    histogram(range)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="KRGG3P"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379,124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379],
"count":[0.0,2.0,4.0,13.0,43.0,100.0,205.0,327.0,477.0,596.0,690.0,666.0,609.0,481.0,373.0,218.0,111.0,57.0,18.0,10.0,1.0,10.0,41.0,103.0,240.0,404.0,663.0,867.0,880.0,752.0,512.0,306.0,135.0,59.0,21.0,4.0,1.0,0.0,1.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("KRGG3P");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

We can customize it like we used to.
From the differences - access to `key` columns, and we can customize the `position`
of bars (within a single x-coordinate), for example — stack them:

In [20]:
groupedRangesDF.plot {
    histogram(range) {
        fillColor(key.category) {
            scale = categorical(listOf(Color.GREEN, Color.ORANGE))
        }
        borderLine.width = 0.0
        width = 1.0
        // Adjust position of bars from different groups
        position = Position.stack()
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MpAsGv"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"values":["#3ba272","#fc8452"]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379,124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379],
"count":[0.0,2.0,4.0,13.0,43.0,100.0,205.0,327.0,477.0,596.0,690.0,666.0,609.0,481.0,373.0,218.0,111.0,57.0,18.0,10.0,1.0,10.0,41.0,103.0,240.0,404.0,663.0,867.0,880.0,752.0,512.0,306.0,135.0,59.0,21.0,4.0,1.0,0.0,1.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"size":0.0,
"sampling":"none",
"width":1.0,
"position":"stack",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("MpAsGv");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

Histogram plot for `GroupBy` (i.e. `GroupBy.histogram(statBinArgs)` extensions) works as well:

In [21]:
groupedRangesDF.histogram("range")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="YpYjZn"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"count":[0.0,2.0,4.0,13.0,43.0,100.0,205.0,327.0,477.0,596.0,690.0,666.0,609.0,481.0,373.0,218.0,111.0,57.0,18.0,10.0,1.0,10.0,41.0,103.0,240.0,404.0,663.0,867.0,880.0,752.0,512.0,306.0,135.0,59.0,21.0,4.0,1.0,0.0,1.0,0.0],
"x":[124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379,124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("YpYjZn");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

... and can be configured the same way:

In [22]:
groupedRangesDF.histogram(binsAlign = BinsAlign.center(500.0)) { x(range) }.configure {
    alpha = 0.6
    // make the bars from different groups overlap with each other
    position = Position.identity()
    // can access key column by name as `String`
    fillColor("category") { scale = categoricalColorBrewer(BrewerPalette.Qualitative.Dark2) }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="uBMfJE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[144.42844599976092,179.9856013997848,215.54275679980873,251.09991219983263,286.65706759985653,322.21422299988046,357.7713783999044,393.3285337999283,428.8856891999522,464.44284459997607,500.0,535.5571554000239,571.1143108000479,606.6714662000718,642.2286216000957,677.7857770001195,713.3429324001435,748.9000878001674,784.4572432001912,820.0143986002151,144.42844599976092,179.9856013997848,215.54275679980873,251.09991219983263,286.65706759985653,322.21422299988046,357.7713783999044,393.3285337999283,428.8856891999522,464.44284459997607,500.0,535.5571554000239,571.1143108000479,606.6714662000718,642.2286216000957,677.7857770001195,713.3429324001435,748.9000878001674,784.4572432001912,820.0143986002151],
"count":[1.0,2.0,11.0,21.0,72.0,145.0,268.0,411.0,547.0,656.0,711.0,612.0,572.0,413.0,276.0,154.0,79.0,34.0,12.0,3.0,5.0,24.0,76.0,163.0,337.0,533.0,820.0,888.0,779.0,654.0,374.0,215.0,89.0,29.0,11.0,2.0,0.0,1.0,0.0,0.0],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_brewer",
"palette":"Dark2"
},{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("uBMfJE");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [23]:
rangesDF.plot {
    groupBy(category) {
        histogram(range)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2wiXgZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"range",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"count":[0.0,2.0,4.0,13.0,43.0,100.0,205.0,327.0,477.0,596.0,690.0,666.0,609.0,481.0,373.0,218.0,111.0,57.0,18.0,10.0,1.0,10.0,41.0,103.0,240.0,404.0,663.0,867.0,880.0,752.0,512.0,306.0,135.0,59.0,21.0,4.0,1.0,0.0,1.0,0.0],
"x":[124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379,124.45004390008367,160.00719930010757,195.5643547001315,231.1215101001554,266.6786655001793,302.23582090020324,337.7929763002271,373.35013170025104,408.90728710027497,444.4644425002989,480.0215979003228,515.5787533003466,551.1359087003706,586.6930641003945,622.2502195004183,657.8073749004423,693.3645303004662,728.9216857004901,764.478841100514,800.0359965005379],
"category":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("2wiXgZ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 range 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 A 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 B